In [2]:
import requests
import json
import pandas as pd
import numpy as np

houses_sold = pd.read_csv('kc_house_data.csv').drop(['id', 'date', 'yr_renovated', 'zipcode', 'sqft_living15', 'sqft_lot15'], axis=1)
houses_for_sale = pd.read_csv('forSale.csv')

In [3]:
houses_for_sale['zip'] = houses_for_sale['address'].str[-5:].astype(int)
kc_zips = pd.read_csv('kc_zips.txt', sep='	')
kc_zips = kc_zips.iloc[:, 0]
houses_for_sale = houses_for_sale[houses_for_sale['zip'].isin(kc_zips)]
houses_for_sale = houses_for_sale.drop('zip', axis=1)
houses_for_sale.describe()
houses_for_sale = houses_for_sale[['price', 'beds', 'baths', 'baths_full', 'baths_half', 'sqft_raw', 'lot_size', 'lat', 'lon']].rename(columns={'beds': 'bedrooms', 'lat': 'latitude', 'lon': 'longitude', 'lot_size': 'sqft_lot', 'sqft_raw': 'sqft', 'price_raw': 'price'})
houses_sold = houses_sold.drop(['floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built'], axis=1).rename(columns={'sqft_living': 'sqft', 'lat': 'latitude', 'long': 'longitude'})
houses_sold.describe()
houses_for_sale['price'] = houses_for_sale['price'].str.replace(r'\D+', '')
houses_for_sale['price'] = houses_for_sale['price'].astype(float)
houses_for_sale.loc[houses_for_sale['sqft_lot'].str.contains('sq ft') == True, 'sqft_lot'] = houses_for_sale['sqft_lot'].str.replace(r'[^0-9.]', '')
houses_for_sale.loc[houses_for_sale['sqft_lot'].str.contains('acres') == True, 'sqft_lot'] = houses_for_sale['sqft_lot'].str.replace(r'[^0-9.]', '').astype(float) * 43560
houses_for_sale['sqft_lot'] = houses_for_sale['sqft_lot'].astype(float)
houses_for_sale = houses_for_sale.dropna()
houses_for_sale.describe()
houses_sold['sqft'] = houses_sold['sqft'].astype(float)
houses_sold['sqft_lot'] = houses_sold['sqft_lot'].astype(float)
houses_sold.drop(houses_sold[houses_sold['bedrooms'] == 0].index, inplace=True)
houses_sold.drop(houses_sold[houses_sold['bathrooms'] < 1].index, inplace=True)
houses_sold = houses_sold.dropna()
houses_sold.drop(houses_sold[houses_sold['bedrooms'] == 33].index, inplace=True)
houses_sold.describe()
houses_for_sale['baths_half'].fillna(0, inplace=True)
houses_for_sale['baths_full'].fillna(0, inplace=True)
houses_for_sale['bathrooms'] = houses_for_sale['baths_full'] + (houses_for_sale['baths_half'] / 2)
houses_for_sale = houses_for_sale[houses_for_sale['bathrooms'] > 0].drop(['baths', 'baths_full', 'baths_half'], axis=1)
houses_for_sale = houses_for_sale[list(houses_sold.columns)]
houses_for_sale.describe()

,price,bedrooms,bathrooms,sqft,sqft_lot,latitude,longitude
count,1.149000e+03,1149.000000,1149.000000,1149.000000,1.149000e+03,1149.000000,1149.000000
mean,1.069199e+06,3.743255,2.724543,2727.362924,2.722189e+04,47.515664,-122.200571
std,1.206985e+06,0.844773,0.873570,1251.900218,1.659849e+05,0.151200,0.119134
min,2.579500e+05,2.000000,1.500000,580.000000,4.360000e+02,47.189501,-122.502297
25%,5.750000e+05,3.000000,2.500000,1950.000000,4.792000e+03,47.373756,-122.295864
50%,7.750000e+05,4.000000,2.500000,2549.000000,7.405000e+03,47.538517,-122.196759
75%,1.200000e+06,4.000000,2.500000,3290.000000,1.176120e+04,47.644254,-122.121324
max,2.000000e+07,7.000000,8.500000,19000.000000,5.096520e+06,47.799142,-121.969077


In [18]:
houses_for_sale.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft', 'sqft_lot', 'latitude',
       'longitude'],
      dtype='object')

In [19]:
houses_for_sale.shape

(1149, 7)

In [24]:
houses_for_sale[houses_for_sale['longitude'] == -122.341617]

,price,bedrooms,bathrooms,sqft,sqft_lot,latitude,longitude


In [25]:
houses_for_sale.dtypes

price        float64
bedrooms       int64
bathrooms    float64
sqft         float64
sqft_lot     float64
latitude     float64
longitude    float64
dtype: object

In [21]:
houses_for_sale.head()

,price,bedrooms,bathrooms,sqft,sqft_lot,latitude,longitude
5,549950.0,2,1.5,1326.0,436.0,47.683442,-122.376457
6,1915000.0,4,3.5,3610.0,6098.0,47.615038,-122.287834
13,700000.0,5,2.5,2860.0,19602.0,47.511299,-122.163993
16,559000.0,4,1.5,1860.0,7841.0,47.501025,-122.161689
20,725000.0,4,3.5,2100.0,6970.0,47.546932,-122.307435


In [26]:
CLIENT_ID = 'EFRCH3WCWDQ3DBHURMC0TGZIRF1AD5X5NPW5DCCEXT1KXPXW' # your Foursquare ID
CLIENT_SECRET = 'WQ4RQAI010LS3XXNR0E0TZNT0M10IM43OQMAWU5LYSWBTP20' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

def getNearbyVenues(latitudes, longitudes, radius=500):
    venues_list=[]
    nearby_venues = pd.DataFrame()
    for lat, lng in zip(latitudes, longitudes):
        print(lat, lng)
        try:
            url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION, 
                lat, 
                lng, 
                radius, 
                100)
            results = requests.get(url).json()["response"]['groups'][0]['items']
            venues_list.append([(
                lat, 
                lng, 
                v['venue']['name'], 
                v['venue']['location']['lat'], 
                v['venue']['location']['lng'],  
                v['venue']['categories'][0]) for v in results])
            nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
            nearby_venues.columns = [ 
                      'Neighborhood Latitude', 
                      'Neighborhood Longitude', 
                      'Venue', 
                      'Venue Latitude', 
                      'Venue Longitude', 
                      'Venue Category']
        except Exception as e:
            print(e)
    return(nearby_venues, venues_list)

# venues, venues_list = getNearbyVenues(houses_sold['latitude'][6000:6500], houses_sold['longitude'][6000:6500])
# venues.to_csv('6500.csv')
venues, venues_list = getNearbyVenues(houses_for_sale['latitude'][800:6500], houses_for_sale['longitude'][800:6500])
venues.to_csv('ven.csv')

47.382583000000004 -122.406869
47.527553999999995 -122.031158
47.435906 -122.16583100000001
47.33777 -122.35179199999999
47.326172 -122.169256
47.609575 -122.309848
47.631190000000004 -121.986415
47.363879 -122.04440600000001
47.251166999999995 -122.19128300000001
47.282076 -122.33896499999999
47.4455 -122.1481
47.4455 -122.1481
47.3143 -122.0163
47.3143 -122.0163
47.316961 -122.37784199999999
47.638178 -122.34178899999999
47.676007 -122.195206
47.4455 -122.1481
47.2648 -122.2476
47.748705 -122.11735800000001
47.48919 -122.352404
47.258145 -122.19989299999999
47.259956 -122.22166599999998
47.4455 -122.1481
47.65440200000001 -122.352979
47.459362 -121.98364
47.319423 -122.198167
47.553957000000004 -122.261757
47.31948 -122.199352
47.521276 -122.35603700000001
47.455767 -122.088975
47.337695000000004 -122.33211399999999
47.671544 -122.18883799999999
47.388158000000004 -122.189085
47.577103 -121.997025
47.368915 -122.045146
47.371755 -122.218451
47.34959600000001 -122.302931
47.492634 -12

In [5]:
df = pd.read_csv("6500.csv")
df.columns

Index(['Unnamed: 0', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

In [6]:
total = pd.read_csv("total.csv")
total.columns

Index(['Unnamed: 0', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Category', 'Venue Latitude', 'Venue Longitude'],
      dtype='object')

In [7]:
df = df[list(total.columns)]

In [10]:
df.shape

(3780, 7)

In [11]:
total.shape

(20974, 7)

In [13]:
df = pd.concat([total, df], ignore_index=True)
df.shape

(45728, 7)

In [15]:
df = df.drop('Unnamed: 0', axis=1)
df

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
0,47.5112,-122.257,Delightful Desserts,"{'id': '4bf58dd8d48988d1d0941735', 'name': 'De...",47.509913,-122.260640
1,47.5112,-122.257,Upper Rainier Beach,"{'id': '52e81612bcbc57f1066b7a11', 'name': 'Gu...",47.507828,-122.256179
2,47.5112,-122.257,Hutchinson Playground,"{'id': '4bf58dd8d48988d1e8941735', 'name': 'Ba...",47.513665,-122.260623
3,47.5112,-122.257,Hutchinson Playground,"{'id': '4bf58dd8d48988d1e7941735', 'name': 'Pl...",47.513404,-122.261846
4,47.7210,-122.319,Java Jane,"{'id': '4bf58dd8d48988d1e0931735', 'name': 'Co...",47.722454,-122.316508
5,47.7210,-122.319,Amante,"{'id': '4bf58dd8d48988d1ca941735', 'name': 'Pi...",47.718363,-122.318383
6,47.7210,-122.319,Seattle Drum School,"{'id': '4bf58dd8d48988d1e9931735', 'name': 'Ro...",47.719770,-122.312640
7,47.7210,-122.319,7-Eleven,"{'id': '4d954b0ea243a5684a65b473', 'name': 'Co...",47.719113,-122.313160
8,47.7210,-122.319,Licorice Fern Natural Area on Thornton Creek,"{'id': '4bf58dd8d48988d163941735', 'name': 'Pa...",47.722444,-122.316559
9,47.7210,-122.319,Mani's Kitchen,"{'id': '4bf58dd8d48988d10f941735', 'name': 'In...",47.718486,-122.318517


In [16]:
df.to_csv("total.csv")

In [28]:
df = pd.read_csv("ven.csv")
df.columns

Index(['Unnamed: 0', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

In [29]:
total = pd.read_csv("forSaleVenues.csv")
total.columns

Index(['Unnamed: 0', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Category', 'Venue Latitude', 'Venue Longitude'],
      dtype='object')

In [30]:
df = df[list(total.columns)]

In [31]:
df.shape

(2902, 7)

In [32]:
total.shape

(17204, 7)

In [33]:
df = pd.concat([total, df], ignore_index=True)
df.shape

(20106, 7)

In [34]:
df = df.drop('Unnamed: 0', axis=1)
df

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,Venue Latitude,Venue Longitude
0,47.730611,-122.172379,Bartell Drugs,"{'id': '4bf58dd8d48988d10f951735', 'name': 'Ph...",47.732060,-122.174353
1,47.730611,-122.172379,Tony's Sports Bar,"{'id': '4bf58dd8d48988d120941735', 'name': 'Ka...",47.731858,-122.176294
2,47.730611,-122.172379,Factory Donuts,"{'id': '4bf58dd8d48988d148941735', 'name': 'Do...",47.730797,-122.173742
3,47.730611,-122.172379,Pizza Bank,"{'id': '4bf58dd8d48988d1ca941735', 'name': 'Pi...",47.731505,-122.176142
4,47.730611,-122.172379,7-Eleven,"{'id': '4d954b0ea243a5684a65b473', 'name': 'Co...",47.731003,-122.174925
5,47.730611,-122.172379,Amy's Nails,"{'id': '4f04aa0c2fb6e1c99f3db0b8', 'name': 'Na...",47.730920,-122.174884
6,47.730611,-122.172379,Kingsgate Ice Arena,"{'id': '4bf58dd8d48988d168941735', 'name': 'Sk...",47.730387,-122.173684
7,47.730611,-122.172379,Chase Bank,"{'id': '4bf58dd8d48988d10a951735', 'name': 'Ba...",47.731664,-122.174987
8,47.730611,-122.172379,Little Caesars Pizza,"{'id': '4bf58dd8d48988d1ca941735', 'name': 'Pi...",47.730167,-122.174332
9,47.730611,-122.172379,Redbox,"{'id': '4bf58dd8d48988d126951735', 'name': 'Vi...",47.730848,-122.174835


In [35]:
df.to_csv("forSaleVenues.csv")